In [1]:
import scipy.io
import numpy as np

from datetime import datetime, timedelta
import time

import tensorflow as tf

import keras
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
from keras import metrics
from keras.models import model_from_json

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import cv2

import pandas as pd

from extract_face_landmarks.py import extract_face_landmarks
from skimage import color
from skimage import io
from skimage.filters import sobel
"""import ray
ray.init(plasma_directory="/workspaces/96273/temp")
import modin.pandas as pd"""

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'import ray\nray.init(plasma_directory="/workspaces/96273/temp")\nimport modin.pandas as pd'

In [2]:
#if you have multiple GPUs, use this block to avoid allocate all GPUs and all memory.

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
from keras import backend as K
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
#VGG-Face for face recognition: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
    
    return vgg_face_descriptor

In [4]:
model = loadVggFaceModel()

In [5]:
#you can download pretrained weights from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [6]:
#open-cv's face detection module
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Read meta data

In [7]:
# Ref https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/
# https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
mat = scipy.io.loadmat('imdb_crop/imdb.mat')

In [8]:
columns = ["dob", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score", "celeb_names", "celeb_id"]

In [9]:
instances = mat['imdb'][0][0][0].shape[1]

In [10]:
df = pd.DataFrame(index = range(0,instances), columns = columns)

In [11]:
df.shape

(460723, 10)

In [12]:
for i in mat:
    if i == "imdb":
        current_array = mat[i][0][0]
        for j in range(len(current_array)):
            #print(j,". ",columns[j],": ",current_array[j][0])
            df[columns[j]] = pd.DataFrame(current_array[j][0])

In [13]:
df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score,celeb_names,celeb_id
0,693726,1968,[01/nm0000001_rm124825600_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1072.926, 161.838, 1214.7839999999999, 303.6...",1.459693,1.118973,['Lee' George Quinones],6488
1,693726,1970,[01/nm0000001_rm3343756032_1899-5-10_1970.jpg],1.0,[Fred Astaire],"[[477.184, 100.352, 622.592, 245.76]]",2.543198,1.852008,['Weird Al' Yankovic],6488
2,693726,1968,[01/nm0000001_rm577153792_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[114.96964308962852, 114.96964308962852, 451....",3.455579,2.985660,[2 Chainz],6488
3,693726,1968,[01/nm0000001_rm946909184_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[622.8855056426588, 424.21750383700805, 844.3...",1.872117,NaN,[50 Cent],6488
4,693726,1968,[01/nm0000001_rm980463616_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1013.8590023603723, 233.8820422075853, 1201....",1.158766,NaN,[A Martinez],6488


In [14]:
#remove pictures does not include face
df = df[df['face_score'] != -np.inf]

#some pictures include more than one face, remove them
df = df[df['second_face_score'].isna()]

#check threshold
df = df[df['face_score'] >= 3]

In [15]:
df.shape #95234

(95234, 10)

In [16]:
def extractNames(name):
    return name[0]

In [17]:
df['celebrity_name'] = df['name'].apply(extractNames)

In [18]:
df.shape

(95234, 11)

In [19]:
#df = df.drop(columns = ['dob', 'photo_taken', 'face_location', 'face_score', 'second_face_score'])

# Load data set images

In [20]:
def getImagePixels(image_path):
    return cv2.imread("imdb_crop/%s" % image_path[0]) #pixel values in scale of 0-255

In [ ]:
tic = time.time()
df['pixels'] = df['full_path'].apply(getImagePixels)
toc = time.time()

print("this block completed in ",toc-tic," seconds...") #562.80 seconds

# Represent images as vectors

In [ ]:
features_list = {}
def findFaceRepresentation(img):
    detected_face = img
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    #-----------------------------
    """
    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x,y,w,h) in faces:
        detected_face = img[int(y):int(y+h), int(x):int(x+w)]

        try:
            margin = 10
            margin_x = int((w * margin)/100); margin_y = int((h * margin)/100)
            detected_face = img[int(y-margin_y):int(y+h+margin_y), int(x-margin_x):int(x+w+margin_x)]
        except:
            print("detected face has no margin")
    """
    #-----------------------------
    
    try: 
        detected_face = cv2.resize(detected_face, (224, 224))
        #plt.imshow(cv2.cvtColor(detected_face, cv2.COLOR_BGR2RGB))
        features = extract_face_landmarks(img[int(y):int(y+h), int(x):int(x+w)])
        left_eye = features[0]
        right_eye = features[1]
        mouth = features[2]
        nose = features[3]
        img = color.rgb2gray(img)
        features_list['original'] = {}
        features_list['original']['le'] = img[left_eye[0], left_eye[1]]
        features_list['original']['re'] = img[right_eye[0], right_eye[1]]
        features_list['original']['mo'] = img[mouth[0], mouth[1]]
        features_list['original']['no'] = img[nose[0], nose[1]]
        img_s = sobel(img)
        features_list['original'] = {}
        features_list['original']['ele'] = img_s[left_eye[0], left_eye[1]]
        features_list['original']['ere'] = img_s[right_eye[0], right_eye[1]]
        features_list['original']['emo'] = img_s[mouth[0], mouth[1]]
        features_list['original']['eno'] = img_s[nose[0], nose[1]]
        
        #normalize detected face in scale of -1, +1

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 127.5
        img_pixels -= 1
        
        representation = model.predict(img_pixels)[0,:]
    except:
        representation = None
        
    return representation  

In [ ]:
tic = time.time()
df['face_vector_raw'] = df['pixels'].apply(findFaceRepresentation) #vector for raw image
toc = time.time()

print("this block completed in ",toc-tic," seconds...")

# Load Your Photo

In [ ]:
img = cv2.imread("sefik.jpg") #pixel values in scale of 0-255
#img = cv2.imread("sefik_2.jpg") #pixel values in scale of 0-255

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

faces = face_cascade.detectMultiScale(img, 1.3, 5)

faces_landmarks = {}

for (x,y,w,h) in faces:
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    features = extract_face_landmarks(img[int(y):int(y+h), int(x):int(x+w)])
    left_eye = features[0]
    right_eye = features[1]
    mouth = features[2]
    nose = features[3]
    img = color.rgb2gray(img)
    features_list['load'] = {}
    features_list['load']['le'] = img[left_eye[0], left_eye[1]]
    features_list['load']['re'] = img[right_eye[0], right_eye[1]]
    features_list['load']['mo'] = img[mouth[0], mouth[1]]
    features_list['original']['no'] = img[nose[0], nose[1]]
    img_s = sobel(img)
    features_list['load'] = {}
    features_list['load']['ele'] = img_s[left_eye[0], left_eye[1]]
    features_list['load']['ere'] = img_s[right_eye[0], right_eye[1]]
    features_list['load']['emo'] = img_s[mouth[0], mouth[1]]
    features_list['load']['eno'] = img_s[nose[0], nose[1]]
    
    
    
    #cv2.rectangle(img,(x,y),(x+w,y+h),(128,128,128),cv2.FILLED)
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    #add 15% margin around the face
    try:
        margin = 10
        margin_x = int((w * margin)/100); margin_y = int((h * margin)/100)
        detected_face = img[int(y-margin_y):int(y+h+margin_y), int(x-margin_x):int(x+w+margin_x)]
    except:
        print("detected face has no margin")
    
    detected_face = cv2.resize(detected_face, (224, 224))

#plt.imshow(detected_face)
#plt.imshow(cv2.cvtColor(detected_face, cv2.COLOR_BGR2RGB))

In [ ]:
img_pixels = image.img_to_array(detected_face)
img_pixels = np.expand_dims(img_pixels, axis = 0)
img_pixels /= 127.5
img_pixels -= 1

In [ ]:
yourself_representation = model.predict(img_pixels)[0,:]

# Find Similarities

In [ ]:
def findCosineSimilarity(source_representation, test_representation=yourself_representation):
    try:
        a = np.matmul(np.transpose(source_representation), test_representation)
        b = np.sum(np.multiply(source_representation, source_representation))
        c = np.sum(np.multiply(test_representation, test_representation))
        return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
    except:
        return 10 #assign a large value. similar faces will have small value.

In [ ]:
def findScore():
    loss = 0
    for i in features_list['original'].keys():
        loss = np.sum(np.abs(features_list['original'][i] - features_list['load'][i]))
    return loss/(len(features_list['original'].keys()))

In [ ]:
df['similarity'] = df['face_vector_raw'].apply(findCosineSimilarity)
print(findScore())

In [ ]:
df = df.sort_values(by=['similarity'], ascending=True)

In [ ]:
"""x = df.iloc[0]['pixels'].reshape(224, 224, 3)/255
plt.imshow(x)"""

In [ ]:
df.head(1)

In [ ]:
#df = df.reset_index()

In [ ]:
#this block might show different pictures of same actors
if True:
    for i in range(0, 7):
        instance = df.iloc[i]
        name = instance['celebrity_name']
        similarity = instance['similarity']
        
        #img = instance['pixels']
        full_path = instance['full_path'][0]
        img = cv2.imread("imdb_crop/%s" % full_path)
        
        print(i,".",name," (",similarity,") - ",full_path)

        plt.axis('off')
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

        print("-------------------------")

In [ ]:
pivot_df = df.drop_duplicates(subset ="celebrity_name")
pivot_df = pivot_df[pivot_df['photo_taken'] >= 2000]

#0: woman, 1: man. if you know the gender of your target image, then you can filter data set and it fasten system
pivot_df = pivot_df[pivot_df['gender'] == 1]

pivot_df = pivot_df.reset_index()

In [ ]:
for i in range(0, 4):
    instance = pivot_df.iloc[i]
    name = instance['celebrity_name']
    similarity = instance['similarity']
    
    similarity = (1 - similarity)*100
    
    #img = instance['pixels']
    full_path = instance['full_path'][0]
    img = cv2.imread("imdb_crop/%s" % full_path)
    
    print(name," (",similarity,"%) - ",full_path)
    
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
    
    print("-------------------------")

In [ ]:
#you can check a specific celebrity
target = pivot_df[pivot_df['celebrity_name'] == 'Jim Parsons']

for index, instance in target.iterrows():
    name = instance['celebrity_name']
    similarity = instance['similarity']
    full_path = instance['full_path'][0]

    similarity = (1 - similarity)*100
    
    print(index,". ", name," (",similarity,") - ",full_path)

    img = cv2.imread("imdb_crop/%s" % full_path)
    
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()